# retreive the 3 documents from openforis sepal github master branch 

In [ ]:
from pathlib import Path 
from component import parameter as cp

# create the data_test folder
data_test_dir = cp.data_test_dir

In [ ]:
data_test_dir.is_dir()

In [ ]:
from urllib.request import urlretrieve

en_json = data_test_dir.joinpath('en.json')
fr_json = data_test_dir.joinpath('fr.json')
es_json = data_test_dir.joinpath('es.json')

urlretrieve ("https://raw.githubusercontent.com/openforis/sepal/master/modules/gui/frontend/src/locale/en/translations.json", en_json)
urlretrieve ("https://raw.githubusercontent.com/openforis/sepal/master/modules/gui/frontend/src/locale/fr/translations.json", fr_json)
urlretrieve ("https://raw.githubusercontent.com/openforis/sepal/master/modules/gui/frontend/src/locale/es/translations.json", es_json)

all((en_json.is_file(), fr_json.is_file(), es_json.is_file()))

# create the jsonline file 

In [ ]:
def get_keys(v, prev_key = None, keys = []):
    if not (isinstance(v, dict) or isinstance(v, list)):
        keys.append(prev_key)
        return keys
    
    new_keys = []
    
    # create an enum object 
    if isinstance(v, dict):
        enum = v.items()
    elif isinstance(v, list):
        enum = enumerate(v)
        
    for k, v2 in enum:
        if prev_key != None:
            new_key = prev_key + [k]
        else:
            new_key = [k]
        new_keys.extend(get_keys(v2, new_key, []))
        
    return new_keys

In [ ]:
import json

json_line = data_test_dir.joinpath('items.txt')

with json_line.open('w') as dst:
    with en_json.open() as en, fr_json.open() as fr, es_json.open() as es:
        
        datasets = {
            'en': json.loads(en.read()),
            'fr': json.loads(fr.read()),
            'es': json.loads(es.read())
        }
        
        items = get_keys(datasets['en'])
        
        for item in items:
            
            line = {'translation': {}}
            
            for local in datasets:
                
                data = datasets[local]
                
                for key in item:
                    data = data[key]
                    
                line['translation'][local] = data
                
            dst.write(f"{json.dumps(line)}\n")